# Training Regression - Reaction

# Import packages

In [ ]:
import os
import sys

current_path=os.getcwd()
print(current_path)

parent_path=os.path.dirname(current_path)
print(parent_path)

if parent_path not in sys.path:
    sys.path.append(parent_path)

In [ ]:
import pandas as pd
from lightning import pytorch as pl
from pathlib import Path

from chemprop import data, featurizers, models, nn

# Change data inputs here

## Load data

In [ ]:
import numpy as np
chemprop_dir = Path.cwd().parent
num_workers = 20  # number of workers for dataloader. 0 means using main process for data loading
# smiles_column = 'AAM'
# target_columns = ['lograte']

## Perform data splitting for training, validation, and testing

## Get ReactionDatasets

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader 
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import MessagePassing, GCNConv, global_mean_pool

# --- BƯỚC 1: ĐỊNH NGHĨA DATASET CHO PRE-TRAINING ---

def get_reverse_edge_index(edge_index):
    """
    Tính toán chỉ số của các cạnh ngược trong một đồ thị vô hướng.
    """
    # Tạo một tensor để lưu kết quả
    rev_edge_index = torch.zeros_like(edge_index[0])
    
    # Tạo một mapping từ cạnh (u, v) tới chỉ số của nó để tra cứu nhanh hơn
    # Chuyển edge_index sang list các tuple để làm key cho dictionary
    edge_map = {tuple(edge.tolist()): i for i, edge in enumerate(edge_index.T)}
    
    for i, edge in enumerate(edge_index.T):
        # Lấy cạnh ngược (v, u)
        rev_edge = tuple(edge.flip(0).tolist())
        # Tìm chỉ số của cạnh ngược trong map
        rev_idx = edge_map[rev_edge]
        rev_edge_index[i] = rev_idx
        
    return rev_edge_index


# 2. CẬP NHẬT LẠI DATASET
class DGIDataset(torch.utils.data.Dataset):
    """
    Dataset này trả về một đối tượng Data của PyG cho mỗi đồ thị,
    với các thuộc tính V, E, và rev_edge_index để tương thích với chemprop.
    """
    def __init__(self, node_attrs, edge_attrs, edge_indices):
        self.node_attrs = node_attrs
        self.edge_attrs = edge_attrs
        self.edge_indices = edge_indices

    def __len__(self):
        return len(self.node_attrs)

    def __getitem__(self, idx):
        # Lấy dữ liệu gốc
        node_tensor = torch.tensor(self.node_attrs[idx], dtype=torch.float32)
        edge_attr_tensor = torch.tensor(self.edge_attrs[idx], dtype=torch.float32)
        edge_index_tensor = torch.tensor(self.edge_indices[idx], dtype=torch.long)
        
        # SỬA Ở ĐÂY: Tính toán và thêm rev_edge_index
        rev_edge_index_tensor = get_reverse_edge_index(edge_index_tensor)

        # Tạo đối tượng Data với đầy đủ các thuộc tính cần thiết
        data = Data(
            V=node_tensor,
            E=edge_attr_tensor,
            edge_index=edge_index_tensor,
            rev_edge_index=rev_edge_index_tensor # Thêm vào đây
        )
        return data

class SimpleGNNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        full().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        # Lưu lại output_dim để mô hình DGI có thể truy cập
        self.output_dim = out_channels

    def forward(self, batch):
        # Lấy các thuộc tính cần thiết từ đối tượng Batch
        x, edge_index = batch.x, batch.edge_index

        # Truyền qua các lớp GNN
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# --- TẢI VÀ KẾT HỢP TẤT CẢ DỮ LIỆU ĐỂ PRE-TRAIN ---
# Pre-training là tự giám sát nên chúng ta có thể dùng cả train/val/test data

# Tải dữ liệu từ các file NPZ của bạn
train_npz = np.load(f'../chemprop/data/RC/full/barriers_rgd1/barriers_rgd1_aam_train_rc_processed_data.npz', allow_pickle=True)
val_npz = np.load(f'../chemprop/data/RC/full/barriers_rgd1/barriers_rgd1_aam_val_rc_processed_data.npz', allow_pickle=True)
test_npz = np.load(f'../chemprop/data/RC/full/barriers_rgd1/barriers_rgd1_aam_test_rc_processed_data.npz', allow_pickle=True)

# Kết hợp dữ liệu
all_node_attrs = np.concatenate((train_npz['node_attrs'], val_npz['node_attrs'], test_npz['node_attrs']))
all_edge_attrs = np.concatenate((train_npz['edge_attrs'], val_npz['edge_attrs'], test_npz['edge_attrs']))
all_edge_indices = np.concatenate((train_npz['edge_indices'], val_npz['edge_indices'], test_npz['edge_indices']))

print(f"Tổng số mẫu để pre-train: {len(all_node_attrs)}")

# def get_reverse_edge_index(edge_index):
#     """Tính toán chỉ số của các cạnh ngược."""
#     rev_edge_index = torch.zeros_like(edge_index[0])
#     for i in range(edge_index.shape[1]):
#         # Tìm cạnh ngược (j, i) cho mỗi cạnh (i, j)
#         edge_to_find = edge_index[:, i].flip(0)
#         # So sánh để tìm vị trí
#         matches = (edge_index.T == edge_to_find).all(dim=1)
#         # Lấy chỉ số đầu tiên tìm thấy
#         rev_idx = torch.where(matches)[0]
#         if rev_idx.numel() > 0:
#             rev_edge_index[i] = rev_idx[0]
#         else:
#             # Xử lý trường hợp không tìm thấy cạnh ngược (ít khả năng xảy ra nếu đồ thị là vô hướng)
#             rev_edge_index[i] = -1 # hoặc một giá trị đặc biệt
#     return rev_edge_index


# def collate_pretraining_batch(samples):
#     """
#     Hàm này sẽ gộp một list các sample (dictionaries) thành một batch duy nhất,
#     đồng thời đảm bảo các thuộc tính V, E, và rev_edge_index được đặt tên đúng.
#     """
#     batch_data = []
#     for i, sample in enumerate(samples):
#         edge_index = sample["edge_index"]
        
#         # *** THÊM BƯỚC NÀY ***
#         # Tính toán rev_edge_index
#         rev_edge_index = get_reverse_edge_index(edge_index)

#         data_point = Data(
#             V=sample["masked_nodes"],
#             E=sample["masked_edges"],
#             edge_index=edge_index,
            
#             # Thêm rev_edge_index vào đối tượng Data
#             rev_edge_index=rev_edge_index,
            
#             original_V=sample["original_nodes"],
#             node_mask_indices=sample["node_mask_indices"],
#             sample_idx=i
#         )
#         batch_data.append(data_point)

#     return Batch.from_data_list(batch_data)

# Khởi tạo lại DataLoader với collate_fn mới
pretrain_dataset = DGIDataset(all_node_attrs, all_edge_attrs, all_edge_indices)
pretrain_loader = DataLoader(pretrain_dataset, batch_size=32, shuffle=True)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message passing

Message passing blocks must be given the shape of the featurizer's outputs.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

## Construct MPNN

In [ ]:
# class Discriminator(nn.Module):
#     """
#     Bộ phân loại (Discriminator) để so sánh embedding của nút và bản tóm tắt đồ thị.
#     """
#     def __init__(self, hidden_dim):
#         super().__init__()
#         # Sử dụng một lớp tuyến tính đơn giản làm hàm tính điểm bilinear
#         self.bilinear = torch.nn.Linear(hidden_dim, hidden_dim, bias=False)

#     def forward(self, node_embeddings, summary_vec):
#         # Tính tích vô hướng giữa embedding nút và summary đã được biến đổi
#         # `sh` = transformed summary
#         sh = self.bilinear(summary_vec)
#         # `logits` có shape [số_nút_trong_batch]
#         logits = torch.sum(node_embeddings * sh, dim=1)
#         return logits


# class DGI_PretrainingMPNN(pl.LightningModule):
#     """
#     Mô hình pre-training cho DGI.
#     """
#     def __init__(self, message_passing):
#         super().__init__()
#         # Lưu lại encoder để có thể truy cập sau này
#         self.message_passing = message_passing
#         hidden_dim = self.message_passing.output_dim
        
#         # 1. Khởi tạo Discriminator
#         self.discriminator = Discriminator(hidden_dim)
        
#         # 2. Khởi tạo hàm loss
#         self.loss_fn = torch.nn.BCEWithLogitsLoss()
        
#         # Lưu lại để có thể gọi trong training_step
#         self.save_hyperparameters(ignore=['message_passing'])

#     def forward(self, batch):
#         """
#         Thực hiện quá trình DGI: tạo embedding, summary, và tính điểm.
#         """
#         # (1) Lấy node embeddings từ encoder
#         # Giả sử message_passing trả về node embeddings có shape [tổng_số_nút, hidden_dim]
#         node_embeddings = self.message_passing(batch)

#         # (2) Tạo bản tóm tắt toàn cục cho mỗi đồ thị trong batch
#         # summary_vec có shape [batch_size, hidden_dim]
#         summary_vec = global_mean_pool(node_embeddings, batch.batch)

#         # (3) Tạo các cặp Dương tính (Positive): (nút, đồ thị của nó)
#         # Mở rộng summary_vec để mỗi nút có một bản sao của summary đồ thị của nó
#         # positive_expanded_summary có shape [tổng_số_nút, hidden_dim]
#         positive_expanded_summary = summary_vec[batch.batch]
        
#         # (4) Tạo các cặp Âm tính (Negative): (nút, đồ thị khác)
#         # "Làm hỏng" (corrupt) các summary bằng cách xáo trộn chúng trong batch
#         # Ví dụ: [s1, s2, s3] -> [s2, s3, s1]
#         corrupted_summary_idx = torch.randperm(summary_vec.size(0))
#         corrupted_summary_vec = summary_vec[corrupted_summary_idx]
#         negative_expanded_summary = corrupted_summary_vec[batch.batch]

#         # (5) Tính điểm cho cả hai loại cặp
#         pos_score = self.discriminator(node_embeddings, positive_expanded_summary)
#         neg_score = self.discriminator(node_embeddings, negative_expanded_summary)

#         return pos_score, neg_score

#     def training_step(self, batch, batch_idx):
#         pos_score, neg_score = self.forward(batch)

#         # Tạo nhãn: 1 cho cặp dương tính, 0 cho cặp âm tính
#         pos_labels = torch.ones_like(pos_score)
#         neg_labels = torch.zeros_like(neg_score)
        
#         # Tính loss
#         loss_pos = self.loss_fn(pos_score, pos_labels)
#         loss_neg = self.loss_fn(neg_score, neg_labels)
#         total_loss = loss_pos + loss_neg

#         self.log("train_loss", total_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         return total_loss

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=1e-3)


In [ ]:
import torch
from torch import Tensor, nn, optim
from torch.nn import MSELoss
from lightning import pytorch as pl

from chemprop.data import BatchMolGraph
from chemprop.nn import MessagePassing
from chemprop.schedulers import build_NoamLike_LRSched

class Discriminator(nn.Module):
    """
    Bộ phân loại (Discriminator) để so sánh embedding của nút và bản tóm tắt đồ thị.
    """
    def __init__(self, hidden_dim):
        super().__init__()
        # Sử dụng một lớp tuyến tính đơn giản làm hàm tính điểm bilinear
        self.bilinear = torch.nn.Linear(hidden_dim, hidden_dim, bias=False)

    def forward(self, node_embeddings, summary_vec):
        # Tính tích vô hướng giữa embedding nút và summary đã được biến đổi
        # `sh` = transformed summary
        sh = self.bilinear(summary_vec)
        # `logits` có shape [số_nút_trong_batch]
        logits = torch.sum(node_embeddings * sh, dim=1)
        return logits


class DGI_PretrainingMPNN(pl.LightningModule):
    """
    Mô hình pre-training cho DGI.
    """
    def __init__(
        self,
        message_passing: MessagePassing,
        # node_feature_dim: int, # THÊM: Cần biết chiều của đặc trưng nút để tái tạo
        warmup_epochs: int = 2,
        init_lr: float = 1e-4,
        max_lr: float = 1e-3,
        final_lr: float = 1e-4,
    ):
        super().__init__()
        self.message_passing = message_passing
        hidden_dim = self.message_passing.output_dim
        
        # 1. Khởi tạo Discriminator
        self.discriminator = Discriminator(hidden_dim)
        
        # 2. Khởi tạo hàm loss
        self.loss_fn = torch.nn.BCEWithLogitsLoss()
        
        # Lưu lại để có thể gọi trong training_step
        self.save_hyperparameters(ignore=['message_passing'])
        
    def forward(self, batch: BatchMolGraph) -> Tensor:
        """
        Thực hiện quá trình DGI: tạo embedding, summary, và tính điểm.
        """
        # (1) Lấy node embeddings từ encoder
        # Giả sử message_passing trả về node embeddings có shape [tổng_số_nút, hidden_dim]
        node_embeddings = self.message_passing(batch)

        # (2) Tạo bản tóm tắt toàn cục cho mỗi đồ thị trong batch
        # summary_vec có shape [batch_size, hidden_dim]
        summary_vec = global_mean_pool(node_embeddings, batch.batch)

        # (3) Tạo các cặp Dương tính (Positive): (nút, đồ thị của nó)
        # Mở rộng summary_vec để mỗi nút có một bản sao của summary đồ thị của nó
        # positive_expanded_summary có shape [tổng_số_nút, hidden_dim]
        positive_expanded_summary = summary_vec[batch.batch]
        
        # (4) Tạo các cặp Âm tính (Negative): (nút, đồ thị khác)
        # "Làm hỏng" (corrupt) các summary bằng cách xáo trộn chúng trong batch
        # Ví dụ: [s1, s2, s3] -> [s2, s3, s1]
        corrupted_summary_idx = torch.randperm(summary_vec.size(0))
        corrupted_summary_vec = summary_vec[corrupted_summary_idx]
        negative_expanded_summary = corrupted_summary_vec[batch.batch]

        # (5) Tính điểm cho cả hai loại cặp
        pos_score = self.discriminator(node_embeddings, positive_expanded_summary)
        neg_score = self.discriminator(node_embeddings, negative_expanded_summary)

        return pos_score, neg_score


    def training_step(self, batch: BatchMolGraph, batch_idx):
        pos_score, neg_score = self.forward(batch)

        # Tạo nhãn: 1 cho cặp dương tính, 0 cho cặp âm tính
        pos_labels = torch.ones_like(pos_score)
        neg_labels = torch.zeros_like(neg_score)
        
        # Tính loss
        loss_pos = self.loss_fn(pos_score, pos_labels)
        loss_neg = self.loss_fn(neg_score, neg_labels)
        total_loss = loss_pos + loss_neg

        self.log("train_loss", total_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return total_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


# Training and testing

## Test results

In [ ]:
# --- BƯỚC 3: KHỞI TẠO VÀ HUẤN LUYỆN MÔ HÌNH PRE-TRAINING (ĐÃ SỬA LỖI) ---
from chemprop import nn
from chemprop.nn import BondMessagePassing
# Lấy thông số chiều từ dữ liệu
node_feature_dim = pretrain_dataset.node_attrs[0].shape[1]
edge_feature_dim = pretrain_dataset.edge_attrs[0].shape[1]
fdims = (node_feature_dim, edge_feature_dim)

# Khởi tạo encoder D-MPNN
mp_encoder = BondMessagePassing(*fdims)
# mp_encoder = nn.Linear(node_feature_dim, 128) # Tạm dùng Linear làm encoder ví dụ
# mp_encoder.output_dim = 128

# hidden_dim = 128
# output_dim = 128 # output_dim của encoder

# # Khởi tạo encoder GNN thực sự
# mp_encoder = SimpleGNNEncoder(
#     in_channels=node_feature_dim,
#     hidden_channels=hidden_dim,
#     out_channels=output_dim
# )
# Khởi tạo mô hình pre-training
# SỬA Ở ĐÂY: Xóa `edge_feature_dim` khỏi lời gọi hàm
pretrain_model = DGI_PretrainingMPNN(
    message_passing=mp_encoder
)

# Khởi tạo Trainer của PyTorch Lightning
trainer = pl.Trainer(
    max_epochs=50, 
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    logger=True
)

# Bắt đầu pre-training!
print("Bắt đầu Pre-training...")
trainer.fit(pretrain_model, pretrain_loader)
print("Pre-training hoàn tất!")


# --- BƯỚC 4: LƯU LẠI ENCODER ĐÃ ĐƯỢC HUẤN LUYỆN ---
output_path = "pretrained_dgi_dmpnn_nhap_encoder.pt"
torch.save(pretrain_model.message_passing.state_dict(), output_path)
print(f"Encoder đã pre-train được lưu tại: {output_path}")